Exemplo retirado do [notebook](https://goo.gle/34cHkDk)

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.13.0


Treinaremos uma rede neural para reconhecer peças de roupa de um conjunto de dados comum chamado Fashion MNIST. Você pode aprender mais sobre este conjunto de dados [here](https://github.com/zalandoresearch/fashion-mnist).

Contém 70.000 peças de roupa em 10 categorias diferentes. Cada peça de roupa está em uma imagem em escala de cinza 28x28. Você pode ver alguns exemplos aqui: ![alt text](https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png)

In [ ]:
mnist = tf.keras.datasets.fashion_mnist

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])

Você notará que todos os valores do número estão entre 0 e 255. Se estivermos treinando uma rede neural, por vários motivos será mais fácil tratarmos todos os valores como entre 0 e 1, um processo chamado ' normalizing'.. .e felizmente em Python é fácil normalizar uma lista como esta sem loop. Você faz assim:

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

**Sequential**: Define uma SEQUÊNCIA de camadas na rede neural

**Flatten**: lembra antes onde nossas imagens eram um quadrado, quando você as imprimiu? Flatten apenas pega esse quadrado e o transforma em um conjunto unidimensional.

**Dense**: Adiciona uma camada de neurônios

Cada camada de neurônios precisa de uma **função de ativação** para lhes dizer o que fazer. Há muitas opções, mas use-as por enquanto.

**Relu** significa efetivamente "Se X>0 retornar X, caso contrário, retornar 0" - então o que ele faz é passar apenas valores 0 ou maiores para a próxima camada da rede.

**Softmax** pega um conjunto de valores e efetivamente escolhe o maior deles, por exemplo, se a saída da última camada for semelhante a [0,1, 0,1, 0,05, 0,1, 9,5, 0,1, 0,05, 0,05, 0,05 ], evita que você pesquise em busca do maior valor e o transforma em [0,0,0,0,1,0,0,0,0] - O objetivo é economizar muita codificação!

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

Depois de concluir o treinamento - você deverá ver um valor de precisão no final da época final. Pode parecer algo como 0,9098. Isso indica que sua rede neural tem cerca de 91% de precisão na classificação dos dados de treinamento. Ou seja, descobriu uma correspondência de padrão entre a imagem e os rótulos que funcionou 91% das vezes. Não é ótimo, mas não é ruim, considerando que foi treinado apenas por 5 épocas e feito rapidamente.

Mas como isso funcionaria com dados invisíveis? É por isso que temos as imagens de teste. Podemos chamar model.evaluate e passar os dois conjuntos, e ele reportará a perda de cada um. Vamos tentar:

In [ ]:
model.evaluate(test_images, test_labels)

Para mim, isso retornou uma precisão de cerca de 0,8838, o que significa que foi cerca de 88% preciso. Como esperado, provavelmente não funcionaria tão bem com dados *invisíveis* quanto com os dados nos quais foi treinado! Ao longo deste curso, você verá maneiras de melhorar isso.

# Exercícios de Exploração

#Exercício 1:
Para este primeiro exercício execute o código abaixo: Ele cria um conjunto de classificações para cada uma das imagens de teste e a seguir imprime a primeira entrada nas classificações. A saída, depois de executada, é uma lista de números. Por que você acha que isso acontece e o que esses números representam?

In [ ]:
classifications = model.predict(test_images)

print(classifications[0])

**Dica**: tente executar print(test_labels[0]) - e você obterá um 9. Isso ajuda você a entender por que esta lista tem a aparência que tem?

In [ ]:
print(test_labels[0])

### O que esta lista representa?


1. São 10 valores aleatórios sem sentido
2. São as 10 primeiras classificações que o computador fez
3. É a probabilidade de este item pertencer a cada uma das 10 classes


### Como você sabe que esta lista indica que o item é uma bota?


1. Não há informações suficientes para responder a essa pergunta
2. O décimo elemento da lista é o maior, e a bota é rotulada como 9
2. A bota tem o rótulo 9 e há 0-> 9 elementos na lista

#Exercício 2:
Vejamos agora as camadas do seu modelo. Experimente valores diferentes para a camada densa com 512 neurônios. Que resultados diferentes você obtém em termos de perda, tempo de treinamento, etc.? Por que você acha que é esse o caso?

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

###Pergunta 1. Aumento para 1.024 neurônios – Qual é o impacto?

1. O treinamento demora mais, mas é mais preciso
2. O treinamento leva mais tempo, mas não afeta a precisão
3. O treinamento leva o mesmo tempo, mas é mais preciso

#Exercício 3:

O que aconteceria se você removesse a camada Flatten(). Por que você acha que é esse o caso?

Você recebe um erro sobre o formato dos dados. Pode parecer vago agora, mas reforça a regra prática de que a primeira camada da sua rede deve ter o mesmo formato dos seus dados. No momento, nossos dados são imagens de 28x28, e 28 camadas de 28 neurônios seriam inviáveis, então faz mais sentido 'achatar' essas 28,28 em 784x1. Em vez de escrevermos todo o código para lidar com isso nós mesmos, adicionamos a camada Flatten() no início e, quando os arrays forem carregados no modelo posteriormente, eles serão automaticamente achatados para nós.

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0


model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# This version has the 'flatten' removed. Replace the above with this one to see the error.
#model = tf.keras.models.Sequential([tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])


model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

#Exercício 4:

Considere as camadas finais (de saída). Por que existem 10 deles? O que aconteceria se você tivesse um valor diferente de 10? Por exemplo, tente treinar a rede com 5

Você recebe um erro assim que encontra um valor inesperado. Outra regra prática: o número de neurônios na última camada deve corresponder ao número de classes para as quais você está classificando. Neste caso são os dígitos de 0 a 9, então há 10 deles, portanto você deve ter 10 neurônios em sua camada final.

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# Replace the above model definiton with this one to see the network with 5 output layers
# And you'll see errors as a result!
# model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
#                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

##Exercício 5:

Considere os efeitos de camadas adicionais na rede. O que acontecerá se você adicionar outra camada entre aquela com 512 e a camada final com 10.

Resposta: Não há um impacto significativo – porque são dados relativamente simples. Para dados muito mais complexos (incluindo imagens coloridas classificadas como flores que você verá na próxima lição), muitas vezes são necessárias camadas extras.

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

#Exercício 6:

Considere o impacto do treinamento por mais ou menos épocas. Por que você acha que seria esse o caso?

Experimente 15 épocas - você provavelmente obterá um modelo com uma perda muito melhor do que aquele com 5
Experimente 30 épocas - você poderá ver que o valor da perda para de diminuir e às vezes aumenta. Este é um efeito colateral de algo chamado 'overfitting', sobre o qual você pode aprender [em algum lugar] e é algo que você precisa observar ao treinar redes neurais. Não adianta perder tempo treinando se você não está melhorando sua perda, certo! :)

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=30)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[34])
print(test_labels[34])

#Exercício 7:

Antes de treinar, você normalizou os dados, passando de valores de 0 a 255 para valores de 0 a 1. Qual seria o impacto de remover isso? Aqui está o código completo para tentar. Por que você acha que obtém resultados diferentes?

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
# To experiment with removing normalization, comment out the following 2 lines
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

#Exercício 8:

Anteriormente, quando você treinou para épocas extras, você teve um problema em que sua perda poderia mudar. Pode ter demorado um pouco para você esperar o treinamento fazer isso e você pode ter pensado 'não seria bom se eu pudesse interromper o treinamento quando atingir o valor desejado?' - ou seja, 95% de precisão pode ser suficiente para você, e se você atingir isso depois de 3 épocas, por que ficar esperando que termine muito mais épocas... Então, como você consertaria isso? Como qualquer outro programa... você tem retornos de chamada! Vamos vê-los em ação...

In [ ]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])




In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                    input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                    input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
